In [1]:
import pandas as pd
df = pd.read_csv("main_database_v4.csv")
df.head()

,season,date,home,score,away,hPos,hGSPG,hGCPG,hCSPG,hPtsPG,...,aGD5,aGD7,hPosLS,aPosLS,hSpent,hIncome,hNet,aSpent,aIncome,aNet
0,9,2000-08-19,charlton,4-0,man city,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,18,18,18.72,0.479,-18.241,18.07,1.320,-16.750
1,9,2000-08-19,chelsea,4-2,west ham,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5,9,47.15,34.830,-12.320,8.92,38.160,29.240
2,9,2000-08-19,coventry,1-3,middlesbrough,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,14,12,17.10,23.650,6.550,29.52,9.000,-20.520
3,9,2000-08-19,derby,2-2,southampton,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,16,15,11.99,6.260,-5.730,0.00,0.131,0.131
4,9,2000-08-19,leeds,2-0,everton,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3,13,47.84,10.920,-36.920,31.03,32.000,0.970


In [11]:
def test_upset(score,hPos,aPos):
    upset = False
    
    if abs(hPos-aPos) >= 7:
        LargeGap = True
    else:
        LargeGap = False
    
    if hPos < aPos:
        HomeBetter = True
    else:
        HomeBetter = False
    
    [home_goals, away_goals] = [x for x in score.split("-")]
    
    if home_goals > away_goals:
        result = "H"
    elif home_goals == away_goals:
        result = "D"
    else:
        result = "A"
        
    if LargeGap:
        if HomeBetter and result != "H":
            upset = True
        elif not(HomeBetter) and result != "A":
            upset = True
    else:
        if HomeBetter and result == "A":
            upset = True
        elif not(HomeBetter) and result == "H":
            upset = True
    if abs(hPos-aPos) <= 2:
        upset = False
    return upset

def upset_function(row,df):
    score = row['score']
    hPos = int(row['hPos'])
    aPos = int(row['aPos'])
    upset = test_upset(score,hPos,aPos)
    if upset:
        season = row['season']
        df_season = df.query("season == @season")
        date = row['date']
        games_so_far = df_season[pd.to_datetime(df_season.date) < pd.to_datetime(date)]
        if len(games_so_far) < 30:
            upset = False
    return upset

In [13]:
df['upset'] = df.apply(upset_function,args=(df,),axis=1)

In [14]:
df.head()

,season,date,home,score,away,hPos,hGSPG,hGCPG,hCSPG,hPtsPG,...,aGD7,hPosLS,aPosLS,hSpent,hIncome,hNet,aSpent,aIncome,aNet,upset
0,9,2000-08-19,charlton,4-0,man city,4.0,0.0,0.0,0.0,0.0,...,0.0,18,18,18.72,0.479,-18.241,18.07,1.320,-16.750,False
1,9,2000-08-19,chelsea,4-2,west ham,5.0,0.0,0.0,0.0,0.0,...,0.0,5,9,47.15,34.830,-12.320,8.92,38.160,29.240,False
2,9,2000-08-19,coventry,1-3,middlesbrough,6.0,0.0,0.0,0.0,0.0,...,0.0,14,12,17.10,23.650,6.550,29.52,9.000,-20.520,False
3,9,2000-08-19,derby,2-2,southampton,7.0,0.0,0.0,0.0,0.0,...,0.0,16,15,11.99,6.260,-5.730,0.00,0.131,0.131,False
4,9,2000-08-19,leeds,2-0,everton,10.0,0.0,0.0,0.0,0.0,...,0.0,3,13,47.84,10.920,-36.920,31.03,32.000,0.970,False


In [15]:
df['upset'].value_counts()

False    4957
True     1883
Name: upset, dtype: int64

In [16]:
print(4957/(4957+1883))

0.7247076023391813


In [ ]:
def get_upset_percentage_so_far(row,df):
    season = row['season']
    df_season = df.query("season == @season")
    date = row['date']
    games_so_far = df_season[pd.to_datetime(df_season.date) < pd.to_datetime(date)]
    
    hPos = row['hPos']
    aPos = row['aPos']